In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv('cleaned.csv', encoding='utf-8')
allowed_labels = ['major', 'minor', 'patch']
labels = df[df['label'].isin(allowed_labels)]

In [65]:
df2= pd.read_csv('cleaned.csv', encoding='utf-8')

In [66]:
df2

,commit_date,api_spec_id,label,externalDocs.added,externalDocs.deleted,Info_contact_name_changes,Servers_modified,Endpoints_added,Endpoints_deleted,Components_parameters_added,...,Info_termsOfService_changes,Components_parameters_deleted,Components_Security_Schemes_modified,Tags_modified,Paths_modified,Security_modified,Security_added,Security_deleted,Endpoints_modified,new
0,2020-10-06 15:12:47+00:00,96.0,NaN,0,0,0,0.0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,0,0,0.0,NaN
1,2020-10-07 07:19:41+00:00,96.0,NaN,0,0,0,0.0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,0,0,0.0,NaN
2,2020-10-07 08:09:45+00:00,96.0,NaN,0,0,0,0.0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,0,0,0.0,NaN
3,2020-10-07 13:54:00+00:00,96.0,NaN,0,0,0,0.0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,0,0,0.0,NaN
4,2020-10-07 15:44:27+00:00,96.0,NaN,0,0,0,0.0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,1,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30118,2020-02-28 15:47:57+00:00,241800.0,NaN,0,0,0,0.0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,0,0,0.0,NaN
30119,2017-05-02 05:59:53+00:00,241801.0,NaN,0,0,0,0.0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,0,0,0.0,NaN
30120,2020-02-28 15:47:57+00:00,241802.0,NaN,0,0,0,0.0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,0,0,0.0,NaN
30121,2020-01-31 01:12:33+00:00,242080.0,NaN,0,0,0,0.0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,0,0,0.0,NaN


In [68]:
labels['label'].value_counts()

patch    1036
minor     703
major     448
Name: label, dtype: int64

In [10]:
import warnings
warnings.filterwarnings('ignore')

# define a dictionary that maps labels to numbers
label_to_number = {'major': 0, 'minor': 1, 'patch': 2}

# use the map() method to apply the mapping to the label column
labels['label'] = labels['label'].map(label_to_number)

In [3]:
labels.fillna(0, inplace=True)

In [38]:
labels.shape

(2187, 44)

In [11]:
# Oversampling using SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

from imblearn.under_sampling import RandomUnderSampler

X = labels.drop(['commit_date', 'label', 'api_spec_id','new'], axis=1)
y = labels['label']   # target variable

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle=True, stratify=y)

oversampled = SMOTE(random_state=0,
                    k_neighbors=5)
X_train_smote, y_train_smote = oversampled.fit_resample(X_train, y_train)

In [58]:
y_train_smote = pd.DataFrame(y_train_smote , columns=['label'])
y_train_smote

,label
0,2
1,0
2,1
3,2
4,0
...,...
2482,1
2483,1
2484,1
2485,1


In [13]:
X_train_smote = pd.DataFrame( X_train_smote, columns=X_train.columns)

In [59]:
X_train_smote.shape

(2487, 40)

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

# Get columns to use
cols_to_use = X_train_smote[['Endpoints_added','Components_parameters_added','Paths_added', 'Paths_deleted', 'Tags_deleted', 'Components_Schemas_Added', 'Info_description_changes', 'Tags_modified', 'Paths_modified', 'Endpoints_modified']].columns

data_scaled = scaler.fit_transform(X_train_smote[cols_to_use])

# Define the model
model = GaussianMixture(n_components=3, random_state=32, max_iter=200, init_params='random', covariance_type='full', n_init=1, warm_start=True)

# Fit the model
model.fit(data_scaled)

# Get the cluster labels
cluster_labels = model.predict(data_scaled)


# Add the predicted labels to the dataframe
y_train_smote['predicted_labels'] = cluster_labels

# Get the accuracy score
accuracy_score(y_train_smote['label'], cluster_labels)

# Get the precision score
precision_score(y_train_smote['label'], cluster_labels, average='weighted')

# Get the recall score
recall_score(y_train_smote['label'], cluster_labels, average='weighted')

# Get the f1 score
f1_score(y_train_smote['label'], cluster_labels, average='weighted')

# Get the classification report
print(classification_report(y_train_smote['label'], cluster_labels))


              precision    recall  f1-score   support

           0       0.21      0.03      0.06       829
           1       0.28      0.64      0.39       829
           2       0.35      0.20      0.25       829

    accuracy                           0.29      2487
   macro avg       0.28      0.29      0.23      2487
weighted avg       0.28      0.29      0.23      2487



In [32]:
y_train_smote['label'].groupby(y_train_smote['predicted_labels']).value_counts()

predicted_labels  label
0                 2         55
                  1         46
                  0         27
1                 0        750
                  2        611
                  1        531
2                 1        252
                  2        163
                  0         52
Name: label, dtype: int64

In [49]:
###  Confusion Matrix for oversampled data

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff

# actual labels and predicted labels
cm = confusion_matrix(y_train_smote['label'], y_train_smote['predicted_labels'])

# plot the confusion matrix

# Create heatmap
fig = ff.create_annotated_heatmap(cm, x=['major', 'minor', 'patch'], y=['major', 'minor', 'patch'], colorscale='brwnyl')

fig.update_layout(
    title='Confusion Matrix for Oversampled Data',
    xaxis_title='Predicted Label',
    yaxis_title='True Label')

fig.show()

In [42]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('cleaned.csv', encoding='utf-8')
allowed_labels = ['major', 'minor', 'patch']
labels = df[df['label'].isin(allowed_labels)]
labels.fillna(0, inplace=True)

X = labels.drop(['api_spec_id', 'label','commit_date','new'], axis=1)
y = labels['label']   # target variable

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle=True, stratify=y)


X_train = pd.DataFrame(X_train , columns= X_train.columns)
y_train = pd.DataFrame(y_train , columns=['label'])


In [44]:
scaler = StandardScaler()

# Get columns to use
cols_to_use = X_train[['Endpoints_added', 'Endpoints_deleted','Components_parameters_added', 'Paths_added', 'Paths_deleted', 'Components_Schemas_Added', 'Components_Schemas_Deleted', 'Info_description_changes', 'Paths_modified', 'Endpoints_modified']].columns

data_scaled = scaler.fit_transform(X_train[cols_to_use])

# Define the model
model = GaussianMixture(n_components=3, random_state=32, max_iter=200, init_params='random', covariance_type='full', n_init=1, warm_start=True)

# Fit the model
model.fit(data_scaled)

# Get the cluster labels
cluster_labels = model.predict(data_scaled)

# Map the cluster labels to major, minor, and patch labels
label_map = {0: "major", 1: "minor", 2: "patch"}
predicted_labels = [label_map[label] for label in cluster_labels]

# Add the major, minor, and patch labels to the dataframe
y_train['predicted_labels'] = predicted_labels

# Get the accuracy score
accuracy_score(y_train['label'], predicted_labels)

# Get the precision score
precision_score(y_train['label'], predicted_labels, average='weighted')

# Get the recall score
recall_score(y_train['label'], predicted_labels, average='weighted')

# Get the f1 score
f1_score(y_train['label'], predicted_labels, average='weighted')

# Get the classification report
print(classification_report(y_train['label'], predicted_labels))



              precision    recall  f1-score   support

       major       0.13      0.03      0.04       358
       minor       0.29      0.69      0.41       562
       patch       0.49      0.21      0.29       829

    accuracy                           0.32      1749
   macro avg       0.30      0.31      0.25      1749
weighted avg       0.35      0.32      0.28      1749



In [48]:
###  Confusion Matrix for cleaned subset of data

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff

# actual labels and predicted labels
cm = confusion_matrix(y_train['label'], y_train['predicted_labels'])

# plot the confusion matrix

# Create heatmap
fig = ff.create_annotated_heatmap(cm, x=['major', 'minor', 'patch'], y=['major', 'minor', 'patch'], colorscale='brwnyl')

fig.update_layout(
    title='Confusion Matrix for Cleaned Subset of Data',
    xaxis_title='Predicted Label',
    yaxis_title='True Label')

fig.show()

### Subset of Data by years

In [51]:
labels['Year'] = pd.to_datetime(labels['commit_date']).dt.year
labels['commit_date'] = pd.to_datetime(labels['commit_date'])

labels_2015 = labels[labels['Year'] == 2015]
labels_2016 = labels[labels['Year'] == 2016]
labels_2017=  labels[labels['Year']== 2017]
labels_2018 = labels[labels['Year'] == 2018]
labels_2019 = labels[labels['Year'] == 2019]
labels_2020 = labels[labels['Year'] == 2020]
labels_2021 = labels[labels['Year'] == 2021]
labels_2022 = labels[labels['Year'] == 2022]


In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import warnings

pd.options.mode.chained_assignment = None


# load subset of dataframes into list dfs
dfs = [labels_2015, labels_2016, labels_2017, labels_2018, labels_2019, labels_2020, labels_2021, labels_2022]

for i, df in enumerate(dfs):
    scaler = StandardScaler()

    # Get columns to use
    cols_to_use = df[['Endpoints_added', 'Components_parameters_added', 'Paths_added', 'Paths_deleted', 'Tags_deleted', 'Components_Schemas_Added', 'Info_description_changes', 'Tags_modified', 'Paths_modified', 'Endpoints_modified']].columns

    data_scaled = scaler.fit_transform(df[cols_to_use])

    # Define the model
    model = GaussianMixture(n_components=3, random_state=32, max_iter=200, init_params='random', covariance_type='full', n_init=1, warm_start=True)

    # Fit the model
    model.fit(data_scaled)

    # Get the cluster labels
    cluster_labels = model.predict(data_scaled)

    # Map the cluster labels to major, minor, and patch labels
    label_map = {0: "major", 1: "minor", 2: "patch"}
    predicted_labels = [label_map[label] for label in cluster_labels]

    # Add the major, minor, and patch labels to the dataframe
    df["predicted_labels"] = predicted_labels

    # Accuracy
    print("Accuracy for dataframe", i+1)
    print(accuracy_score(df["label"], df["predicted_labels"]))
    
    # Precision
    print("Precision for dataframe", i+1)
    print(precision_score(df["label"], df["predicted_labels"], average="weighted"))

    # Recall
    print("Recall for dataframe", i+1)
    print(recall_score(df["label"], df["predicted_labels"], average="weighted"))

    # F1
    print("F1 for dataframe", i+1)
    print(f1_score(df["label"], df["predicted_labels"], average="weighted"))

    # Classification report
    print(classification_report(df["label"], df["predicted_labels"]))



Accuracy for dataframe 1
0.1875
Precision for dataframe 1
0.05000000000000001
Recall for dataframe 1
0.1875
F1 for dataframe 1
0.07894736842105264
              precision    recall  f1-score   support

       major       0.00      0.00      0.00        35
       minor       0.20      0.75      0.32        12
       patch       0.00      0.00      0.00         1

    accuracy                           0.19        48
   macro avg       0.07      0.25      0.11        48
weighted avg       0.05      0.19      0.08        48

Accuracy for dataframe 2
0.32941176470588235
Precision for dataframe 2
0.30427807486631014
Recall for dataframe 2
0.32941176470588235
F1 for dataframe 2
0.23529411764705888
              precision    recall  f1-score   support

       major       0.17      0.06      0.09        33
       minor       0.35      0.96      0.51        24
       patch       0.43      0.11      0.17        28

    accuracy                           0.33        85
   macro avg       0.31    

In [57]:
# Compute confusion matrix for each dataframe using plotly

from sklearn.metrics import confusion_matrix
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go

for i, df in enumerate(dfs):
    # Compute confusion matrix
    cm = confusion_matrix(df["label"], df["predicted_labels"])

    # Create heatmap
    fig = ff.create_annotated_heatmap(cm, x=['major', 'minor', 'patch'], y=['major', 'minor', 'patch'], colorscale='brwnyl')

    # Add title
    fig.update_layout(
    title_text="Confusion matrix for Dataframe {}".format(i+1),
    xaxis=dict(title='Predicted Labels'),
    yaxis=dict(title='True Labels')
)
    # Show heatmap
    fig.show()
